In [95]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

In [96]:
news_600_author = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/news_author/news_600_author.csv')


In [97]:
news_600_author_1 = news_600_author.copy()

In [101]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

def add_credit(dataframe_author):
    author_list = list(dataframe_author["author"])
    credit_article = ["None"] * len(author_list)
    credit_followers = ["None"] * len(author_list)
    credit_tweets = ["None"] * len(author_list)
    for i, names in enumerate(tqdm(author_list)):
        if names == "NaN":
            continue
        elif "NBA.com" in str(names) or "NBA News" in str(names) or "Press" in str(names) or "Official Release" in str(names) or "NBA Twitter" in str(names):
            credit_article[i] = "Default"
            credit_followers[i] = "Default"
            credit_tweets[i] = "Default"
        else:
            try:
                player_search = str(names).replace(" ", "-")
                url = "https://muckrack.com/" + player_search
                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                html = soup.find_all('a', attrs={'class':"profile-edit-btn"})
                if len(html) == 0:
                    num_articles = "None"
                    credit_article[i] = num_articles
                else:
                    for line in html:
                        if "See all" in line.text:
                            span_elem = line.find('span')
                            text_content = span_elem.text.strip()
                            num_articles = text_content.split()[0].replace(',', '')
                            credit_article[i] = num_articles
                            break
                    if credit_followers[i] == "None":
                        credit_followers[i] = "None"
                        
                    tweet = soup.find('p', attrs={'class':"fs-6 mt-0 mb-6"})
                    followers = tweet.find('strong').text.replace(',', '')
                    tweets = tweet.find_all('strong')[1].text.replace(',', '')
                    if len(followers) != 0:
                        credit_followers[i] = followers
                    if len(followers) == 0:
                        followers = "0" 
                        credit_followers[i] = followers
                    if len(tweets) != 0:
                        credit_tweets[i] = tweets
                    if len(tweets) == 0:
                        tweets = "0"
                        credit_tweets[i] = tweets
            except Exception as e:
                if credit_followers[i] == "None":
                    credit_followers[i] = "None"
                if credit_tweets[i] == "None":
                    credit_tweets[i] = "None"
    dataframe_author["credit_article"] = credit_article
    dataframe_author["credit_followers"] = credit_followers
    dataframe_author["credit_tweets"] = credit_tweets
    return dataframe_author


In [102]:
news_600_author_cedit = add_credit(news_600_author_1)

100%|██████████| 6000/6000 [06:01<00:00, 16.59it/s]  


In [103]:
news_600_author_cedit

,Unnamed: 0,title,time,url,author,credit_article,credit_followers,credit_tweets
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...","October 24, 2020",http://global.nba.com/news/bucks-george-hill-r...,Steve Megargee,118532,1896,33433
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...","October 24, 2020",http://global.nba.com/news/nba-wnba-ramp-up-ef...,Brian Martin,199,202,1153
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...","October 23, 2020",http://global.nba.com/news/pat-riley-discusses...,Tim Reynolds | Associated Press,Default,Default,Default
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,"October 23, 2020",http://global.nba.com/news/report-amare-stoude...,NBA News,Default,Default,Default
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,"October 22, 2020",http://global.nba.com/news/espn-to-host-virtua...,Official Release,Default,Default,Default
...,...,...,...,...,...,...,...,...
5995,5995,"Antetokounmpo Scores 34, Bucks Top Spurs 135-129","November 24, 2018",http://global.nba.com/news/antetokounmpo-score...,NBA News,Default,Default,Default
5996,5996,Mavericks Pull Away In 4th For 113-104 Win Ove...,"November 24, 2018",http://global.nba.com/news/mavericks-pull-away...,NBA News,Default,Default,Default
5997,5997,"Murray Leads Nuggets Past Thunder, 105-98","November 24, 2018",http://global.nba.com/news/murray-leads-nugget...,NBA News,Default,Default,Default
5998,5998,"Towns’ 35 Points, 22 Boards Lead Wolves Over B...","November 24, 2018",http://global.nba.com/news/towns-35-points-22-...,NBA News,Default,Default,Default


In [110]:
news_600_author_cedit.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/author_credit_nba/news_600_author_credit.csv', index=False)
